In [1]:
import sys
sys.path.append('/Users/Ziqi/Desktop/developer/GWR-UI-ZIQI/pysal/contrib/gwr/')
from pysal.contrib.gwr.gwr import MGWR,GWR
from pysal.contrib.glm.glm import GLM
from pysal.contrib.gwr.sel_bw import Sel_BW
#sys.path.append('/Users/Ziqi/Downloads/')

import numpy as np
import pandas as pd

In [2]:
#df = pd.read_csv('/Users/Ziqi/Desktop/2016 Election Data/df_z_cleaned_cols.csv')
#With 3rd_party included
df = pd.read_csv('/Users/Ziqi/Desktop/developer/GWR-UI-ZIQI/test.csv') 

In [10]:
y = df.z_pct_dem.as_matrix().reshape(-1,1)
df['Intercept'] = 1
X = df.ix[:,5:].as_matrix()
u = df.proj_X
v = df.proj_Y
coords = list(zip(u,v))

In [63]:
%%time
bw = Sel_BW(coords, y, X, multi=True,constant=False)
bws = bw.search(criterion='AICc',tol_multi=1e-05, rss_score=True,init_multi=False)
XB = bw.XB
err = bw.err 
results = MGWR(coords, y, X, bws, XB, err,constant=False).fit()


scipy.stats.pearsonr(y,results.predy)

(99, 2)


NameError: name 'scipy' is not defined

In [73]:
import scipy
scipy.stats.pearsonr(y,results.predy)[0]
yhat = results.predy
np.sum((y-np.mean(y))/np.std(y)*(yhat-np.mean(yhat))/np.std(yhat))/99

0.65149423112471783

In [92]:
np.sum(np.multiply(y-np.mean(y),yhat-np.mean(yhat)))/np.std(y)/np.std(yhat)

64.49792888134705

In [91]:
np.multiply(y-np.mean(y),yhat-np.mean(yhat))

array([[-0.29315617],
       [ 0.01207311],
       [-0.27683924],
       [ 1.38148119],
       [ 0.75779422],
       [-1.50578947],
       [ 0.28881549],
       [ 0.08230598],
       [-0.10805414],
       [ 0.7911703 ],
       [-0.05372274],
       [ 0.30881462],
       [ 0.54312169],
       [ 0.47197721],
       [ 0.7734445 ],
       [ 0.1131515 ],
       [ 0.12980462],
       [ 0.18169746],
       [ 0.01484398],
       [ 0.15143284],
       [ 0.08762804],
       [ 0.55974916],
       [-0.0164041 ],
       [ 3.39560171],
       [ 0.425575  ],
       [-0.10812906],
       [ 0.15084302],
       [ 0.43198731],
       [ 0.48866381],
       [ 0.15256419],
       [ 0.45864876],
       [ 3.04353564],
       [ 1.74589636],
       [ 0.16180773],
       [ 0.08844669],
       [ 0.71018119],
       [ 0.24592222],
       [ 0.17697019],
       [ 0.18204675],
       [ 0.26639915],
       [ 0.14573061],
       [ 0.17005714],
       [ 1.52031129],
       [ 3.3798244 ],
       [-0.08808132],
       [ 1

In [81]:
scipy.stats.pearsonr(y,results.predy)[0]

array([ 0.65149423])

In [68]:
pMean = np.mean(results.params[1,],axis=0)
pMin = np.min(results.params,axis=0)
pMax = np.max(results.params,axis=0)
pMedian = np.median(results.params,axis=0)
pSTD = np.std(results.params,axis=0)
pQ3, pQ1 = np.percentile(results.params, [75 ,25],axis=0)
IQR = pQ3 - pQ1
rSTD = IQR/1.349
pSummary = pd.DataFrame(np.column_stack((pMean,pSTD,pMin,pQ1,pMedian,pQ3,pMax,IQR,rSTD)))
pSummary.columns = ['Mean','STD','Min','Q1','Median','Q3','Max','IQR','robust_STD']
print (pSummary.to_csv(sep=' ', index=True, header=True))

 Mean STD Min Q1 Median Q3 Max IQR robust_STD
0 -0.25822337016976893 0.5964452986820763 -1.1931268546958944 -0.7400358109399049 -0.2847919607296049 0.3185463960307199 0.5581794152975565 1.0585822069706248 0.7847162394148441
1 -0.9578131731760681 0.4509951421596054 -1.8606936714412783 -1.2708910962267481 -0.9060090694076267 -0.7125130253333605 -0.08443245248292974 0.5583780708933876 0.41391999324936074



In [300]:
#GWR
bw1 = Sel_BW(coords, y, X,constant=False,fixed=False).search(criterion='AICc', search='golden_section',
                        bw_min=3.0, bw_max=200)

bw2 = Sel_BW(coords, y, X,constant=False,fixed=False).search(criterion='AICc', search='golden_section')

bw3 = Sel_BW(coords, y, X,constant=False,fixed=False).search(criterion='AICc', search='interval',
                        bw_min=80.0, bw_max=95.0, interval=3)

results = GWR(coords, y, X, bw1,kernel='bisquare',constant=True).fit()
print(bw1,bw2,bw3)

83.0 83.0 80.0


In [327]:
GLM(y,X).fit().params.shape

(12,)

In [113]:
k = 12
n = result.n
aicc = -2*result.llf + 2.0*k*(k+1)/(n-k-1)  
aicc

111.38901349022663

In [107]:
aic = -2*result.llf + 2*k
aic

131.76110651348245

In [117]:
v1 = result.df_model
n = result.model.n
-2 * result.llf + 2.0*v1*(v1 + 1.0)/(n-v1-1.0)

110.79558927210313

In [349]:
#np.repeat([1,2], 5, axis=1)
est = np.array([GLM(y,X).fit().params]*99)
est_gwr = results.params
XB = np.multiply(est_gwr, X)


In [289]:
import pyproj

xCoor = df[['X']].ix[:,0].tolist()
yCoor = df[['Y']].ix[:,0].tolist()
wgs84=pyproj.Proj("+init=EPSG:4326")
web=pyproj.Proj("+init=EPSG:3857")
xx, yy = pyproj.transform(wgs84, web, xCoor, yCoor)
scoords = list(zip(xx,yy))

In [204]:
np.allclose(squareform(pdist(coords)),distMat)

True